# DATA TRANSFORMATION - 5_Bio_Asma
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [22]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'Data/'

## 1. Load

In [23]:
modulo = 'Biologicos Asma'
dcc = gf.diccionario_llaves()

In [24]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,fecha,Nombre_Producto_Mvto,Nom_Generico,Cantidad,year,month,year_month
0,1853167,2019-08-02,OMALIZUMAB 150 MG POLV INY (XOLAIR)-21755-4896,OMALIZUMAB,2,2019,8,201908
1,695889,2019-07-31,OMALIZUMAB 150 MG POLV INY (XOLAIR)-21755-4896,OMALIZUMAB,2,2019,7,201907
2,1534187,2019-01-05,OMALIZUMAB SOLUCIÓN INYECTABLE 150MG /1ML (XOL...,OMALIZUMAB,4,2019,1,201901
3,1831103,2018-12-03,OMALIZUMAB SOLUCIÓN INYECTABLE 150MG /1ML (XOL...,OMALIZUMAB,3,2018,12,201812
4,1831103,2019-01-03,OMALIZUMAB SOLUCIÓN INYECTABLE 150MG /1ML (XOL...,OMALIZUMAB,3,2019,1,201901


In [25]:
# rename columns
base.rename( columns={ 'Nom_Generico':'generico'}, inplace=True)

# Elimina columnas sobran
base = base.drop(columns=['Nombre_Producto_Mvto'])

# Cambiar texto a minuscula.
base = gf.letra_lower(base,'generico')

# Elimina dplicados
base = base.drop_duplicates()

In [26]:
# Cantidades positivas.

# base = base[base['Cantidad']<0]
# base = base[base['generico']=='omalizumab']
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12198 entries, 0 to 12593
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          12198 non-null  int64         
 1   fecha       12198 non-null  datetime64[ns]
 2   generico    12198 non-null  object        
 3   Cantidad    12198 non-null  int64         
 4   year        12198 non-null  int64         
 5   month       12198 non-null  int64         
 6   year_month  12198 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 762.4+ KB


In [28]:
base_ = base.groupby(['id','year','month','generico'])['Cantidad'].sum().\
    reset_index( name = 'dosis' )
base_

,id,year,month,generico,dosis
0,500547,2019,7,omalizumab,1
1,500588,2017,12,omalizumab,3
2,500588,2018,1,omalizumab,9
3,500588,2018,2,omalizumab,6
4,500588,2018,3,omalizumab,3
...,...,...,...,...,...
8810,1999629,2020,1,dupilumab,2
8811,1999629,2020,1,omalizumab,2
8812,1999629,2020,2,dupilumab,1
8813,1999629,2020,3,dupilumab,1


In [33]:
# pivot table to have separate variables, nans replaced with zero
base_p = base_.pivot_table(index=['id','year','month'], columns='generico', values=['dosis']).reset_index()
base_p.columns = ['_'.join(col).strip() for col in base_p.columns.values]
base_p.fillna( 0 , inplace = True)
base_p.head()

,id_,year_,month_,dosis_benralizumab,dosis_dupilumab,dosis_mepolizumab,dosis_omalizumab
0,500547,2019,7,0.0,0.0,0.0,1.0
1,500588,2017,12,0.0,0.0,0.0,3.0
2,500588,2018,1,0.0,0.0,0.0,9.0
3,500588,2018,2,0.0,0.0,0.0,6.0
4,500588,2018,3,0.0,0.0,0.0,3.0


In [34]:
base_p.rename( columns={ 'id_':'id', 'year_':'year', 'month_':'month' }, inplace=True)

## 3. Merge with ids table`

In [35]:
# 2016 - 2020
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_hab = ids_mensual.merge( base_p, how='left')
base_final_hab.head()

,id,year,month,dosis_benralizumab,dosis_dupilumab,dosis_mepolizumab,dosis_omalizumab
0,500547,2016,1,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN


In [36]:
base_final_hab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53280 entries, 0 to 53279
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  53280 non-null  int64  
 1   year                53280 non-null  int64  
 2   month               53280 non-null  int64  
 3   dosis_benralizumab  8976 non-null   float64
 4   dosis_dupilumab     8976 non-null   float64
 5   dosis_mepolizumab   8976 non-null   float64
 6   dosis_omalizumab    8976 non-null   float64
dtypes: float64(4), int64(3)
memory usage: 3.3 MB
